# Learning Week 5

Questions provided here: 
https://work.caltech.edu/homework/hw5.pdf

In [1]:
# Load modules
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from __future__ import division
from __future__ import print_function

## Question 5

In [21]:
# Function to optimize
def error_func (w):
    u,v = w[0], w[1]
    return (u*np.exp(v) - 2*v*np.exp(-u))**2
# Partial derivatives
def grad(w):
    u,v = w[0], w[1]
    partial_u = 2*(u*np.exp(v) - 2*v*np.exp(-u))*(np.exp(v) + 2*v*np.exp(-u))
    partial_v = 2*(u*np.exp(v) - 2*v*np.exp(-u))*(u*np.exp(v) - 2*np.exp(-u))
    return np.array([partial_u, partial_v])

def gradient_descent(iters):
    # Initialize at (1,1)
    learning_rate = 0.1
    w = np.array([1,1])
    print ("Initial weights: {}".format(w))
    print ("Initial error: {}".format(error_func(w)))
    
    # Iterate
    for i in range(iters):
        w = w - learning_rate*grad(w)
        print ("Iteration {}".format(i+1))
        print ("\tCurrent weights: {}".format(w))
        print ("\tCurrent error: {}".format(error_func(w)))
    
gradient_descent(10)

Initial weights: [1 1]
Initial error: 3.93039723188
Iteration 1
	Current weights: [-0.36954299  0.21392055]
	Current error: 1.15950972997
Iteration 2
	Current weights: [ 0.03052069 -0.50793405]
	Current error: 1.00740748296
Iteration 3
	Current weights: [ 0.10752311 -0.12221026]
	Current error: 0.0990091216273
Iteration 4
	Current weights: [ 0.06564483 -0.01516656]
	Current error: 0.00866064536281
Iteration 5
	Current weights: [ 0.04784117  0.0184899 ]
	Current error: 0.000181755791728
Iteration 6
	Current weights: [ 0.04499946  0.02349925]
	Current error: 1.29723984784e-06
Iteration 7
	Current weights: [ 0.04475602  0.0239243 ]
	Current error: 7.29152469846e-09
Iteration 8
	Current weights: [ 0.04473775  0.02395617]
	Current error: 4.00999789056e-11
Iteration 9
	Current weights: [ 0.04473639  0.02395854]
	Current error: 2.20168344841e-13
Iteration 10
	Current weights: [ 0.04473629  0.02395871]
	Current error: 1.20868339442e-15


## Question 7

In [23]:
def coordinate_descent(iters):
    # Initialize at (1,1)
    learning_rate = 0.1
    w = np.array([1,1])
    print ("Initial weights: {}".format(w))
    print ("Initial error: {}".format(error_func(w)))
    
    # Iterate
    for i in range(iters):
        # Step along u
        w = w - learning_rate * np.array([grad(w)[0], 0])
        # Step along v
        w = w - learning_rate * np.array([0, grad(w)[1]])
        print ("Iteration {}".format(i+1))
        print ("\tCurrent weights: {}".format(w))
        print ("\tCurrent error: {}".format(error_func(w)))

coordinate_descent(15)

Initial weights: [1 1]
Initial error: 3.93039723188
Iteration 1
	Current weights: [-0.36954299 -2.03992288]
	Current error: 34.2901631123
Iteration 2
	Current weights: [ 6.39247376 -2.17869889]
	Current error: 0.534142591372
Iteration 3
	Current weights: [ 6.37699523 -2.28347262]
	Current error: 0.432660827324
Iteration 4
	Current weights: [ 6.36460749 -2.36821379]
	Current error: 0.365039735019
Iteration 5
	Current weights: [ 6.35427659 -2.43959481]
	Current error: 0.31646807536
Iteration 6
	Current weights: [ 6.34542086 -2.50137528]
	Current error: 0.279763423064
Iteration 7
	Current weights: [ 6.33767806 -2.55589957]
	Current error: 0.250986311675
Iteration 8
	Current weights: [ 6.33080628 -2.60473485]
	Current error: 0.227783298944
Iteration 9
	Current weights: [ 6.32463553 -2.64898351]
	Current error: 0.208656695724
Iteration 10
	Current weights: [ 6.31904159 -2.689452  ]
	Current error: 0.192605658614
Iteration 11
	Current weights: [ 6.31393077 -2.72674885]
	Current error: 0.1789

Coordinate descent performs substantially worse than the plain gradient descent algorithm.